In [1]:
#imported all the libraries used

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.model_selection import KFold

In [2]:
#mounted drive to upload csv file

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#path defined from which csv file is read
path="/content/drive/MyDrive/small_data.csv"
data = pd.read_csv(path)

path_cards="/content/drive/MyDrive/sd254_cards.csv"
data_cards = pd.read_csv(path_cards)

path_users="/content/drive/MyDrive/sd254_users.csv"
data_users = pd.read_csv(path_users)
#One-hot encoding for categorical feature 'Sex'
#data = pd.get_dummies(data, columns = ['Sex'])

#Displayed datatypes and null/non-null values
print("Info on transactions")
print(data.info())

print()
print("Info on cards")
print(data_cards.info())

print()
print("Info on users")
print(data_users.info())
#Displayed feature statistics
print()
print("Transactions Statistics")
display(data.describe())

print()
print("Cards Statistics")
display(data_cards.describe())

print()
print("User Statistics")
display(data_users.describe())


#Labeled all features and target
#data = data[['Sex_F', 'Sex_I', 'Sex_M', 'Length', 'Diameter', 'Height', 'Whole Weight', 'Shucked Weight', 'Viscera Weight', 'Shell Weight', 'Rings']]

#Scaled data - mean = 0, var = 1
#scaler = StandardScaler()
#data[['Length', 'Diameter', 'Height', 'Whole Weight', 'Shucked Weight', 'Viscera Weight', 'Shell Weight']] = scaler.fit_transform(data[['Length', 'Diameter', 'Height', 'Whole Weight', 'Shucked Weight', 'Viscera Weight', 'Shell Weight']])

In [ ]:
data

In [ ]:
data_cards

In [ ]:
data_users

In [ ]:
print(data['Is Fraud?'].value_counts())
print()
print(data["Errors?"].value_counts())
print()
#data["Use Chip"].value_count()
print(data_cards["Card Brand"].value_counts())
print()
print(data_cards["Card Type"].value_counts())
print()
print(data_cards["Card on Dark Web"].value_counts())

In [ ]:
data = data.drop(['Unnamed: 0', 'Month', 'Day', 'Time', 'Merchant Name', 'Merchant City', 'Merchant State', 'Zip'], axis = 1)
data_cards = data_cards.drop(['Card Number', 'Expires', 'CVV', 'Has Chip', 'Cards Issued', 'Acct Open Date', 'Card on Dark Web'], axis = 1)
data_users = data_users.drop(['Person', 'Retirement Age', 'Birth Month', 'Birth Year',  'Address', 'Apartment', 'City', 'State','Zipcode' ,'Latitude', 'Longitude', 'Per Capita Income - Zipcode', 'Num Credit Cards'], axis=1)

In [ ]:
data['Amount'] = data['Amount'].apply(lambda x: x[1:])
data['Amount'] = pd.to_numeric(data['Amount'])

data_cards['Credit Limit'] = data_cards['Credit Limit'].apply(lambda x: x[1:])
data_cards['Credit Limit'] = pd.to_numeric(data_cards['Credit Limit'])

data_users['Yearly Income - Person'] = data_users['Yearly Income - Person'].apply(lambda x: x[1:])
data_users['Yearly Income - Person'] = pd.to_numeric(data_users['Yearly Income - Person'])
data_users['Total Debt'] = data_users['Total Debt'].apply(lambda x: x[1:])
data_users['Total Debt'] = pd.to_numeric(data_users['Total Debt'])

In [ ]:
encoder = LabelEncoder()

data = data.fillna('')
data['Errors?']= encoder.fit_transform(data['Errors?'])
data['Use Chip']= encoder.fit_transform(data['Use Chip'])
data['Is Fraud?']= encoder.fit_transform(data['Is Fraud?'])

data_cards = data_cards.fillna('')
data_cards['Card Brand']= encoder.fit_transform(data_cards['Card Brand'])
data_cards['Card Type']= encoder.fit_transform(data_cards['Card Type'])

data_users = data_users.fillna('')
data_users['Gender']= encoder.fit_transform(data_users['Gender'])

In [ ]:
data_users['User'] = data_users.index

In [ ]:
data = data.join(data_cards.set_index(['User', 'CARD INDEX']), on=['User', 'Card'])
data = data.join(data_users.set_index('User'), on='User')
data

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
data

In [ ]:
#Box plots and distribution curves of all features

fig, ax = plt.subplots(16, 2, figsize = (20, 20))
sns.boxplot(x= data["User"], ax = ax[0,0])
sns.distplot(data['User'], ax = ax[0,1])
sns.boxplot(x= data["Card"], ax = ax[1,0])
sns.distplot(data['Card'], ax = ax[1,1])
sns.boxplot(x= data["Year"], ax = ax[2,0])
sns.distplot(data['Year'], ax = ax[2,1])
sns.boxplot(x= data["Amount"], ax = ax[3,0])
sns.distplot(data['Amount'], ax = ax[3,1])
sns.boxplot(x= data["Use Chip"], ax = ax[4,0])
sns.distplot(data['Use Chip'], ax = ax[4,1])
sns.boxplot(x= data["MCC"], ax = ax[5,0])
sns.distplot(data['MCC'], ax = ax[5,1])
sns.boxplot(x= data["Errors?"], ax = ax[6,0])
sns.distplot(data['Errors?'], ax = ax[6,1])
sns.boxplot(x= data["Card Brand"], ax = ax[7,0])
sns.distplot(data['Card Brand'], ax = ax[7,1])
sns.boxplot(x= data["Card Type"], ax = ax[8,0])
sns.distplot(data['Card Type'], ax = ax[8,1])
sns.boxplot(x= data["Credit Limit"], ax = ax[9,0])
sns.distplot(data['Credit Limit'], ax = ax[9,1])
sns.boxplot(x= data["Year PIN last Changed"], ax = ax[10,0])
sns.distplot(data['Year PIN last Changed'], ax = ax[10,1])
sns.boxplot(x= data["Current Age"], ax = ax[11,0])
sns.distplot(data['Current Age'], ax = ax[11,1])
sns.boxplot(x= data["Gender"], ax = ax[12,0])
sns.distplot(data['Gender'], ax = ax[12,1])
sns.boxplot(x= data["Yearly Income - Person"], ax = ax[13,0])
sns.distplot(data['Yearly Income - Person'], ax = ax[13,1])
sns.boxplot(x= data["Total Debt"], ax = ax[14,0])
sns.distplot(data['Total Debt'], ax = ax[14,1])
sns.boxplot(x= data["FICO Score"], ax = ax[15,0])
sns.distplot(data['FICO Score'], ax = ax[15,1])

plt.tight_layout()

In [ ]:
#Distribution curves of target

plot = sns.distplot(data['Is Fraud?'])
plot.figure.savefig("2.PNG")

In [ ]:
corr = data.corr()
dataplot = sb.heatmap(corr, cmap="YlGnBu")
data = data.iloc[:,:]
dataplot.figure.savefig("3.PNG")

In [ ]:
data = data[['User', 'Card', 'Year',	'Amount',	'Use Chip',	'MCC',	'Errors?',	'Card Brand',	'Card Type',	'Credit Limit',	'Year PIN last Changed',	'Current Age',	'Gender',	'Yearly Income - Person',	'Total Debt',	'FICO Score', 	'Is Fraud?']]
data

In [ ]:
np_data = np.array(data)
X = np_data
Y = np_data
X = X[:,0: (X.shape[1] - 1)]
Y = Y[:, X.shape[1]]

X = np.c_[np.ones(X.shape[0]), X]

X_train = X[0: int(0.8*X.shape[0]), :]
X_test = X[int(0.8*X.shape[0]):, :]
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:int(0.8*Y.shape[0]), :]
Y_test = Y[int(0.8*Y.shape[0]):, :]

# **First cell below** - 
Standardize features by removing the mean and scaling to unit variance.

The standard score of a sample x is calculated as:

z = (x - u) / s

where u is the mean of the training samples or zero if with_mean=False, and s is the standard deviation of the training samples or one if with_std=False.

# **Second cell below** - 
Transform features by scaling each feature to a given range.

This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

The transformation is given by:

X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
where min, max = feature_range.

# **Third cell below** - 
Scale features using statistics that are robust to outliers.

This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).

Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. Median and interquartile range are then stored to be used on later data using the transform method.

In [ ]:
np_data = np.array(data)
X_mm = np_data
Y_mm = np_data
X_mm = X_mm[:,0: (X_mm.shape[1] - 1)]
Y_mm = Y_mm[:, X_mm.shape[1]]

scaler = MinMaxScaler()
X_mm = scaler.fit_transform(X_mm)

X_mm = np.c_[np.ones(X_mm.shape[0]), X_mm]

X_train_mm = X_mm[0: int(0.8*X_mm.shape[0]), :]
X_test_mm = X_mm[int(0.8*X_mm.shape[0]):, :]
Y_mm = Y_mm.reshape(Y_mm.shape[0], 1)
Y_train_mm = Y_mm[0:int(0.8*Y_mm.shape[0]), :]
Y_test_mm = Y_mm[int(0.8*Y_mm.shape[0]):, :]

In [ ]:
np_data = np.array(data)
X_ss = np_data
Y_ss = np_data
X_ss = X_ss[:,0: (X_ss.shape[1] - 1)]
Y_ss = Y_ss[:, X_ss.shape[1]]

scaler = StandardScaler()
X_ss = scaler.fit_transform(X_ss)

X_ss = np.c_[np.ones(X_ss.shape[0]), X_ss]

X_train_ss = X_ss[0: int(0.8*X_ss.shape[0]), :]
X_test_ss = X_ss[int(0.8*X_ss.shape[0]):, :]
Y_ss = Y_ss.reshape(Y_ss.shape[0], 1)
Y_train_ss = Y_ss[0:int(0.8*Y_ss.shape[0]), :]
Y_test_ss = Y_ss[int(0.8*Y_ss.shape[0]):, :]

In [ ]:
np_data = np.array(data)
X_r = np_data
Y_r= np_data
X_r = X_r[:,0: (X_r.shape[1] - 1)]
Y_r = Y_r[:, X_r.shape[1]]

scaler = RobustScaler()
X_r = scaler.fit_transform(X_r)

X_r = np.c_[np.ones(X_r.shape[0]), X_r]

X_train_r = X_r[0: int(0.8*X_r.shape[0]), :]
X_test_r = X_r[int(0.8*X_r.shape[0]):, :]
Y_r = Y_r.reshape(Y_r.shape[0], 1)
Y_train_r = Y_r[0:int(0.8*Y_r.shape[0]), :]
Y_test_r = Y_r[int(0.8*Y_r.shape[0]):, :]

In [ ]:
clf = LogisticRegression().fit(X_train, Y_train)
clf_mm = LogisticRegression().fit(X_train_mm, Y_train_mm)
clf_ss = LogisticRegression().fit(X_train_ss, Y_train_ss)
clf_r = LogisticRegression().fit(X_train_r, Y_train_r)

In [ ]:
print("For training - ")
print(clf.score(X_train, Y_train))
print(clf_mm.score(X_train_mm, Y_train_mm))
print(clf_ss.score(X_train_ss, Y_train_ss))
print(clf_r.score(X_train_r, Y_train_r))
print()
print("For testing - ")
print(clf.score(X_test, Y_test))
print(clf_mm.score(X_test_mm, Y_test_mm))
print(clf_ss.score(X_test_ss, Y_test_ss))
print(clf_r.score(X_test_r, Y_test_r))

In [ ]:
Y_pred = clf.predict(X_test)
Y_pred_mm = clf_mm.predict(X_test_mm)
Y_pred_ss = clf_ss.predict(X_test_ss)
Y_pred_r = clf_r.predict(X_test_r)


In [ ]:
first = 0
second = 0
third = 0
fourth = 0

#For No preprocessing
#Computed True positives, False positives, False Negatives and True Negatives

total_pred = Y_pred.shape[0]
for i in range(Y_pred.shape[0]):
    if Y_pred[i] == Y_test[i][0] and Y_test[i][0] == 1.0:
        first = first + 1
    if Y_pred[i] != Y_test[i][0] and Y_test[i][0] == 1.0:
        second = second + 1
    if Y_pred[i] != Y_test[i][0] and Y_test[i][0] == 0.0:
        third = third + 1
    if Y_pred[i] == Y_test[i][0] and Y_test[i][0] == 0.0:
        fourth = fourth + 1

#Constructed confusion matrix
confusion_matrix = np.array([[first, second], [third, fourth]])

#Computed Accuracy, Precision, Recall and f_1 Score and printed them
precision = first/(first+second)
recall = first/(first+third)
print("Confusion matrix (No pre processing):")
print(str(confusion_matrix))
print("Accuracy: ", str((first+fourth)/(first+second+third+fourth) * 100), "%")
print("Precision: ", str(precision))
print("Recall: ", str(recall))
print("f1_Score: ", str(2*((precision*recall)/(precision+recall))))

#For max min scaled data
#Computed True positives, False positives, False Negatives and True Negatives

first = 0
second = 0
third = 0
fourth = 0

total_pred = Y_pred_mm.shape[0]
for i in range(Y_pred_mm.shape[0]):
    if Y_pred_mm[i] == Y_test_mm[i][0] and Y_test_mm[i][0] == 1.0:
        first = first + 1
    if Y_pred_mm[i] != Y_test_mm[i][0] and Y_test_mm[i][0] == 1.0:
        second = second + 1
    if Y_pred_mm[i] != Y_test_mm[i][0] and Y_test_mm[i][0] == 0.0:
        third = third + 1
    if Y_pred_mm[i] == Y_test_mm[i][0] and Y_test_mm[i][0] == 0.0:
        fourth = fourth + 1

#Constructed confusion matrix
confusion_matrix = np.array([[first, second], [third, fourth]])

#Computed Accuracy, Precision, Recall and f_1 Score and printed them
precision = first/(first+second)
recall = first/(first+third)
print()
print("Confusion matrix (Max Min scaling):")
print(str(confusion_matrix))
print("Accuracy: ", str((first+fourth)/(first+second+third+fourth) * 100), "%")
print("Precision: ", str(precision))
print("Recall: ", str(recall))
print("f1_Score: ", str(2*((precision*recall)/(precision+recall))))

#####

first = 0
second = 0
third = 0
fourth = 0

#For Standard scaling data
#Computed True positives, False positives, False Negatives and True Negatives

total_pred = Y_pred_ss.shape[0]
for i in range(Y_pred_ss.shape[0]):
    if Y_pred_ss[i] == Y_test_ss[i][0] and Y_test_ss[i][0] == 1.0:
        first = first + 1
    if Y_pred_ss[i] != Y_test_ss[i][0] and Y_test_ss[i][0] == 1.0:
        second = second + 1
    if Y_pred_ss[i] != Y_test_ss[i][0] and Y_test_ss[i][0] == 0.0:
        third = third + 1
    if Y_pred_ss[i] == Y_test_ss[i][0] and Y_test_ss[i][0] == 0.0:
        fourth = fourth + 1

#Constructed confusion matrix
confusion_matrix = np.array([[first, second], [third, fourth]])

#Computed Accuracy, Precision, Recall and f_1 Score and printed them
precision = first/(first+second)
recall = first/(first+third)
print()
print("Confusion matrix (Standard scaling)):")
print(str(confusion_matrix))
print("Accuracy: ", str((first+fourth)/(first+second+third+fourth) * 100), "%")
print("Precision: ", str(precision))
print("Recall: ", str(recall))
print("f1_Score: ", str(2*((precision*recall)/(precision+recall))))

#For robust scaling
#Computed True positives, False positives, False Negatives and True Negatives

first = 0
second = 0
third = 0
fourth = 0

total_pred = Y_pred_r.shape[0]
for i in range(Y_pred_r.shape[0]):
    if Y_pred_r[i] == Y_test_r[i][0] and Y_test_r[i][0] == 1.0:
        first = first + 1
    if Y_pred_r[i] != Y_test_r[i][0] and Y_test_r[i][0] == 1.0:
        second = second + 1
    if Y_pred_r[i] != Y_test_r[i][0] and Y_test_r[i][0] == 0.0:
        third = third + 1
    if Y_pred_r[i] == Y_test_r[i][0] and Y_test_r[i][0] == 0.0:
        fourth = fourth + 1

#Constructed confusion matrix
confusion_matrix = np.array([[first, second], [third, fourth]])

#Computed Accuracy, Precision, Recall and f_1 Score and printed them
precision = first/(first+second)
recall = first/(first+third)
print()
print("Confusion matrix (Robust scaling):")
print(str(confusion_matrix))
print("Accuracy: ", str((first+fourth)/(first+second+third+fourth) * 100), "%")
print("Precision: ", str(precision))
print("Recall: ", str(recall))
print("f1_Score: ", str(2*((precision*recall)/(precision+recall))))


In [ ]:
fig.savefig("1.PNG")

In [ ]:
pred_y=clf.predict(X_test) 

probs_y=clf.predict_proba(X_test) 

precision, recall, thresholds = precision_recall_curve(Y_test, probs_y[:, 1]) 
   #retrieve probability of being 1(in second column of probs_y)
pr_auc = auc(recall, precision)

plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
precision1, recall1, thresholds1 = precision_recall_curve(Y_test, probs_y[:, 0])
fig1, ax1 = plt.subplots()
ax1.plot(precision, recall, label = 'Fraud')
ax1.plot(precision1, recall1, label = 'Not Fraud')
ax1.legend()
ax1.set_title("ROC Curves")
ax1.set_xlabel("False Positive Rate")
ax1.set_ylabel("True Positive Rate")


In [ ]:
base_model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
param_grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = KFold(n_splits = 5)

grid_search_recall = GridSearchCV(estimator=base_model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='recall',error_score=0)
grid_result_recall = grid_search_recall.fit(X_train, Y_train)

grid_search_precision = GridSearchCV(estimator=base_model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='precision',error_score=0)
grid_result_precision = grid_search_precision.fit(X_train, Y_train)

grid_search_accuracy = GridSearchCV(estimator=base_model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result_accuracy = grid_search_accuracy.fit(X_train, Y_train)


In [ ]:
score_arr_recall = grid_search_recall.cv_results_['mean_test_score'].reshape(len(c_values), len(solvers))
score_arr_recall = score_arr_recall.T
c_values_str = list(map(str, c_values))
print("Recall scores")
print(score_arr_recall)

score_arr_precision = grid_search_precision.cv_results_['mean_test_score'].reshape(len(c_values), len(solvers))
score_arr_precision = score_arr_precision.T
c_values_str = list(map(str, c_values))
print("Precision scores")
print(score_arr_precision)


score_arr_accuracy = grid_search_accuracy.cv_results_['mean_test_score'].reshape(len(c_values), len(solvers))
score_arr_accuracy = score_arr_accuracy.T
c_values_str = list(map(str, c_values))
print("Accuracy scores")
print(score_arr_accuracy)

In [ ]:

fig, ax = plt.subplots(1,3, figsize = (15,4))
fig.suptitle('Recall Plots')
ax[0].bar(c_values_str, score_arr_recall[0], label = '1')
ax[0].set_title('Solver - Newton-cg')
ax[0].set_xlabel('C-Values')
ax[0].set_ylabel('Score')
ax[1].bar(c_values_str, score_arr_recall[1], label = '1')
ax[1].set_title('Solver - lbfgs')
ax[1].set_xlabel('C-Values')
ax[1].set_ylabel('Score')
ax[2].bar(c_values_str, score_arr_recall[2], label = '1')
ax[2].set_title('Solver - liblinear')
ax[2].set_xlabel('C-Values')
ax[2].set_ylabel('Score')

fig, ax = plt.subplots(1,3, figsize = (15,4))
fig.suptitle('Precision Plots')
ax[0].bar(c_values_str, score_arr_precision[0], label = '1')
ax[0].set_title('Solver - Newton-cg')
ax[0].set_xlabel('C-Values')
ax[0].set_ylabel('Score')
ax[1].bar(c_values_str, score_arr_precision[1], label = '1')
ax[1].set_title('Solver - lbfgs')
ax[1].set_xlabel('C-Values')
ax[1].set_ylabel('Score')
ax[2].bar(c_values_str, score_arr_precision[2], label = '1')
ax[2].set_title('Solver - liblinear')
ax[2].set_xlabel('C-Values')
ax[2].set_ylabel('Score')

fig, ax = plt.subplots(1,3, figsize = (15,4))
fig.suptitle('Accuracy Plots')
ax[0].bar(c_values_str, score_arr_accuracy[0], label = '1')
ax[0].set_title('Solver - Newton-cg')
ax[0].set_xlabel('C-Values')
ax[0].set_ylabel('Score')
ax[1].bar(c_values_str, score_arr_accuracy[1], label = '1')
ax[1].set_title('Solver - lbfgs')
ax[1].set_xlabel('C-Values')
ax[1].set_ylabel('Score')
ax[2].bar(c_values_str, score_arr_accuracy[2], label = '1')
ax[2].set_title('Solver - liblinear')
ax[2].set_xlabel('C-Values')
ax[2].set_ylabel('Score')


In [ ]:
print("Best recall: %f using %s" % (grid_result_recall.best_score_, grid_result_recall.best_params_))
means_recall = grid_result_recall.cv_results_['mean_test_score']
stds_recall = grid_result_recall.cv_results_['std_test_score']
params_recall = grid_result_recall.cv_results_['params']

print("Best precision: %f using %s" % (grid_result_precision.best_score_, grid_result_precision.best_params_))
means_precision = grid_result_precision.cv_results_['mean_test_score']
stds_precision = grid_result_precision.cv_results_['std_test_score']
params_precision = grid_result_precision.cv_results_['params']

print("Best accuracy: %f using %s" % (grid_result_accuracy.best_score_, grid_result_accuracy.best_params_))
means_accuracy = grid_result_accuracy.cv_results_['mean_test_score']
stds_accuracy = grid_result_accuracy.cv_results_['std_test_score']
params_accuracy = grid_result_accuracy.cv_results_['params']

print()
print("Best model: Solver: %s, C_value: %f" % (solvers[0], c_values[4]))
print("Accuracy: %f, Precision: %f, Recall: %f" % (score_arr_accuracy[0][4], score_arr_precision[0][4], score_arr_recall[0][4]))